# DIY Covid-19 Dashboard

The COVID 19 Pandemic is an ongoing global pandemic happening since 2019. Since december 2020 vaccines have been approved and widely distributed. This dashboard aims to analyse the impacts of the pandemic interms of the number of hospital cases and the useage of ventilators.

In [11]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [12]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [13]:

with open("covid_test_UK_Hospital.json","rt") as INFILE:
    data = json.load(INFILE)
jsondata={}

In [14]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(data):
    data_list = data['data']
    date_dictionary= [dictionary['date'] for dictionary in data_list]
    date_dictionary.sort()
    start_date = parse_date(date_dictionary[0])
    end_date = parse_date(date_dictionary[-1])
    index=pd.date_range(start_date,end_date,freq='D')
    covid_uk_datadf = pd.DataFrame(index=index, columns =["patients_hospital","mechanical_ven"])
    for entry in data_list: 
        dataentry=entry['date']
        for column in ["patients_hospital","mechanical_ven"]:
            if pd.isna(covid_uk_datadf.loc[dataentry, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0# 
                covid_uk_datadf.loc[dataentry, column]=value
    return covid_uk_datadf

df=wrangle_data(data) 
df

,patients_hospital,mechanical_ven
2020-03-27,7267.0,0.0
2020-03-28,8278.0,0.0
2020-03-29,9525.0,0.0
2020-03-30,11093.0,0.0
2020-03-31,12597.0,0.0
...,...,...
2022-11-13,6597.0,0.0
2022-11-14,6603.0,0.0
2022-11-15,6446.0,0.0
2022-11-16,6273.0,0.0


In [15]:

def access_api():
    filters = ['areaType=overview']
    structure = {"date":"date",
            "patients_hospital":"hospitalCases",
            "mechanical_ven":"covidOccupiedMVBeds",
            "new_cases":"newCasesByPublishDate",
            "fully_vac":"cumPeopleVaccinatedCompleteByVaccinationDate"}
    api = Cov19API(filters=filters,structure=structure)
    covid_test=api.get_json()
    apibutton.icon="check"
    apibutton.disabled=True
    return covid_test

In [16]:

def api_button_callback(button):
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    apibutton.icon="check"
 

    
apibutton=wdg.Button(
    description='Refresh', 
    disabled=False,
    button_style='info', 
    tooltip="Keep calm and order pineapple pizza",
    icon='info'
)

apibutton.on_click(api_button_callback) 
display(apibutton)


Button(button_style='info', description='Refresh', icon='info', style=ButtonStyle(), tooltip='Keep calm and or…

## Graphs and Analysis

This graph compares the number of hospital cases as well as the number of covid occupied ventilator beds

In [17]:
def plot_random_walk(walk):
    df[walk].plot()
    plt.show() 
    
analysis_cols = wdg.SelectMultiple(
options =["patients_hospital","mechanical_ven"],
value=["patients_hospital","mechanical_ven"],
rows=2,
description="Hosptial Cases|Use of ventilators",
disabled=False)

scale =wdg.RadioButtons(
options = ['linear','log'],
descripion="Scale:",
disabled=False
)

controler = wdg.HBox([analysis_cols,scale])

def refresh_graph():
    current=analysis_cols.value
    if current==analysis_cols.options[0]:
        other=analysis_cols.options[1]
    else:
        other=analysis_cols.options[0]
    analysis_cols.value=other 
    analysis_cols.value=current


def analysis_graph(gcols,gscale):
    if gscale == 'linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
    
    
graph=wdg.interactive_output(analysis_graph, {'gcols': analysis_cols, 'gscale': scale})

display(controler, graph)

Output()

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*